Object Relational Mapper API of SQLAlchemy Tutorial by Tutorials Point

DATE : 27.10.2022

https://www.tutorialspoint.com/sqlalchemy/index.htm

The main objective of the Object Relational Mapper API of SQLAlchemy is to facilitate associating user-defined Python classes with database tables, and objects of those classes with rows in their corresponding tables. Changes in states of objects and rows are synchronously matched with each other. SQLAlchemy enables expressing database queries in terms of user defined classes and their defined relationships.


In [7]:
from sqlalchemy import Table, Column, Integer, String, MetaData, create_engine
engine = create_engine('sqlite:///sales.db', echo = False)

A base class stores a catlog of classes and mapped tables in the Declarative system. This is called as the declarative base class. There will be usually just one instance of this base in a commonly imported module. The declarative_base() function is used to create base class. This function is defined in sqlalchemy.ext.declarative module.

In [8]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [9]:
class Customers(Base):
   __tablename__ = 'customers'
   
   id = Column(Integer, primary_key = True)
   name = Column(String)
   address = Column(String)
   email = Column(String)

In [10]:
Base.metadata.create_all(engine)

In order to interact with the database, we need to obtain its handle. A session object is the handle to database. Session class is defined using sessionmaker() – a configurable session factory method which is bound to the engine object created earlier.

In [11]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind = engine)

session = Session()

In [12]:
# Add a customer
c1 = Customers(name = 'Emre Kaynar', address = 'Sancaktepe', email = 'ek@gmail.com')
session.add(c1)

# Commit to session
session.commit()


In [13]:
# Add multiple objects
session.add_all([
   Customers(name = 'Eren Mungan', address = 'Anadolu', email = 'em@gmail.com'), 
   Customers(name = 'Emre Acır', address = 'Avrupa', email = 'ea@gmail.com')]
)

session.commit()

In [14]:
# Query objects are initially generated using the query() method of the Session
result = session.query(Customers).all()

for row in result:
   print (row.name, row.address, row.email)

Emre Kaynar Sancaktepe ek@gmail.com
Eren Mungan Anadolu em@gmail.com
Emre Acır Avrupa ea@gmail.com


In [17]:
x = session.query(Customers).get(1)

print(x.name, x.address, x.email)

Emre Kaynar Sancaktepe ek@gmail.com


In [27]:
x.address = "Sancaktepe, Akpınar"

In [29]:
x = session.query(Customers).first()

print ("Name: ", x.name, "Address:", x.address, "Email:", x.email)


Name:  Emre Kaynar Address: Sancaktepe, Akpınar Email: ek@gmail.com


In [30]:
session.rollback()

print ("Name: ", x.name, "Address:", x.address, "Email:", x.email)

Name:  Emre Kaynar Address: Sancaktepe Email: ek@gmail.com


In [33]:
session.query(Customers).filter(Customers.id != 2).update({Customers.name:"Mr."+Customers.name}, synchronize_session = False)


# Database file is updated after commitment
session.commit()

In [37]:
# Using LIKE operator

result = session.query(Customers).filter(Customers.name.like('Mr%'))
for row in result:
   print ("ID:", row.id, "Name: ",row.name, "Address:",row.address, "Email:",row.email)

ID: 1 Name:  Mr.Mr.Emre Kaynar Address: Sancaktepe Email: ek@gmail.com
ID: 3 Name:  Mr.Mr.Emre Acır Address: Avrupa Email: ea@gmail.com


In [39]:
# Using all()
result = session.query(Customers).all()

for row in result:
   print ("ID:", row.id, "Name: ",row.name, "Address:",row.address, "Email:",row.email)

ID: 1 Name:  Mr.Mr.Emre Kaynar Address: Sancaktepe Email: ek@gmail.com
ID: 2 Name:  Eren Mungan Address: Anadolu Email: em@gmail.com
ID: 3 Name:  Mr.Mr.Emre Acır Address: Avrupa Email: ea@gmail.com


In [40]:
from sqlalchemy import text

result = session.query(Customers).from_statement(text("SELECT * FROM customers")).all()

for row in result:
   print ("ID:", row.id, "Name: ",row.name, "Address:",row.address, "Email:",row.email)

ID: 1 Name:  Mr.Mr.Emre Kaynar Address: Sancaktepe Email: ek@gmail.com
ID: 2 Name:  Eren Mungan Address: Anadolu Email: em@gmail.com
ID: 3 Name:  Mr.Mr.Emre Acır Address: Avrupa Email: ea@gmail.com


In [43]:
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship

class Invoice(Base):
   __tablename__ = 'invoices'
   
   id = Column(Integer, primary_key = True)
   custid = Column(Integer, ForeignKey('customers.id'))
   invno = Column(Integer)
   amount = Column(Integer)
   customer = relationship("Customers", back_populates = "invoices")

Customers.invoices = relationship("Invoice", order_by = Invoice.id, back_populates = "customers")
Base.metadata.create_all(engine)

In [44]:
c1 = Customers(name = "Gopal Krishna", address = "Bank Street Hydarebad", email = "gk@gmail.com")
c1.invoices = [Invoice(invno = 10, amount = 15000), Invoice(invno = 14, amount = 3850)]

session.add(c1)
session.commit()

In [ ]:
for c, i in session.query(Customers, Invoice).filter(Customers.id == Invoice.custid).all():
   print ("ID: {} Name: {} Invoice No: {} Amount: {}".format(c.id,c.name, i.invno, i.amount))